<a href="https://colab.research.google.com/github/jbottum/t5pat/blob/main/score2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install einops
!pip install transformers
!pip install --upgrade transformers
!pip install accelerate
!pip install torch
!pip install langchain
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00


In [3]:
import time
import matplotlib.pyplot as plt
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline
import transformers
import torch

import bert_score
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)


import os
# Disable parallelism and avoid the warning message
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
# create a sample model data using python variables and lists
user_id = 'jbottum'
project_id = 'project1'
model_id = 'google/flan-t5-large'

# Define prompts and question types
prompts = [
    'What is the capital of Germany?',
    'What is the capital of Spain?',
    'What is the capital of Canada?',
    'What is the next number in the sequence: 2, 4, 6, 8, ...? If all cats have tails, and Fluffy is a cat, does Fluffy have a tail?',
    'If you eat too much junk food, what will happen to your health? How does smoking affect the risk of lung cancer?',
    'In the same way that pen is related to paper, what is fork related to? If tree is related to forest, what is brick related to?',
    'Every time John eats peanuts, he gets a rash. Does John have a peanut allergy? Every time Sarah studies for a test, she gets an A. Will Sarah get an A on the next test if she studies?',
    'All dogs have fur. Max is a dog. Does Max have fur? If it is raining outside, and Mary does not like to get wet, will Mary take an umbrella?',
    'If I had studied harder, would I have passed the exam? What would have happened if Thomas Edison had not invented the light bulb?',
    'The center of Tropical Storm Arlene, at 02/1800 UTC, is near 26.7N 86.2W. This position is about 425 km/230 nm to the west of Fort Myers in Florida, and it is about 550 km/297 nm to the NNW of the western tip of Cuba. The tropical storm is moving southward, or 175 degrees, 4 knots. The estimated minimum central pressure is 1002 mb. The maximum sustained wind speeds are 35 knots with gusts to 45 knots. The sea heights that are close to the tropical storm are ranging from 6 feet to a maximum of 10 feet.  Precipitation: scattered to numerous moderate is within 180 nm of the center in the NE quadrant. Isolated moderate is from 25N to 27N between 80W and 84W, including parts of south Florida.  Broad surface low pressure extends from the area of the tropical storm, through the Yucatan Channel, into the NW part of the Caribbean Sea.   Where and when will the storm make landfall?',
]

types = [
    'Knowledge Retrieval',
    'Knowledge Retrieval',
    'Knowledge Retrieval',
    'Logical Reasoning',
    'Cause and Effect',
    'Analogical Reasoning',
    'Inductive Reasoning',
    'Deductive Reasoning',
    'Counterfactual Reasoning',
    'In Context'
]

# Example reference answers
reference_list = [
    'The capital of Germany is Berlin',
    'The capital of Spain is Madrid',
    'The capital of Canada is Ottawa',
    'The next number in the sequence is 10.  Yes, Fluffy is a cat and therefore has a tail.',
    'Eating junk food can result in health problems like weight gain and high cholesterol. Smoking can cause lung issues including cancer.',
    'Fork is related to a plate.  A brick is related to a building.',
    'Maybe, to determine if Johns rash is caused by peanuts, he should take an allergy test for peanuts.   Maybe, Sarah will likely do well if she studies and she may be able to get an A.',
    'Yes, Max is a dog and has fur.   Yes, Mary will take an umbrella.',
    'Yes, if you studied harder, you would have passed the test.  If Thomas Edison did not invent the light blub, another inventor would have created the light bulb.',
    'If Arlene continues in the same direciton and speed, storm will make landfall in the Forida Keys in 18 hours from this report.'
]



In [ ]:
# load sample model data into a python dictionary
data = {
    "user_id": user_id,
    "project_id": project_id,
    "model_id": model_id,
    "prompts": prompts,
    "types": types,
    "reference_list": reference_list
}


In [ ]:
# load the sample data into a JSON File

import json

# Save data as JSON
with open("data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)


In [ ]:
# load the sample data into a CSV file

import csv

# Transpose the data to convert it to CSV format
transposed_data = list(zip(prompts, types, reference_list))

# Save data as CSV
with open("data.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Prompts", "Types", "Reference List"])
    writer.writerows(transposed_data)


JSON Schema for the Data:

{
  "user_id": "string",
  "project_id": "string",
  "model_id": "string",
  "prompts": [
    "string",
    ...
  ],
  "types": [
    "string",
    ...
  ],
  "reference_list": [
    "string",
    ...
  ]
}


Explanation:

user_id (string): The user_id field is a string that should contain the unique identifier of the user who owns the project. This identifier helps in associating the data with a specific user account or project owner.

project_id (string): The project_id field is a string that should contain the unique identifier of the project associated with the data. It allows users to differentiate between various projects and their corresponding data.

model_id (string): The model_id field is a string that should contain the identifier of the model used for the project. This identifier helps users identify the specific language model or AI model used for processing the prompts.

prompts (list of strings): The prompts field is a list of strings. Each string in this list represents a different type of question or scenario that the model is expected to answer or process. Users should provide the prompts in the order they want them processed.

types (list of strings): The types field is a list of strings. Each string in this list corresponds to the type of the prompt in the same position in the prompts list. The types categorize the prompts based on their nature, such as "Knowledge Retrieval," "Logical Reasoning," etc.

reference_list (list of strings): The reference_list field is a list of strings. Each string in this list corresponds to the example reference answer for the prompt in the same position in the prompts list. These reference answers represent the expected model responses for each prompt.

Note:

The ... in the schema represents that the respective lists (prompts, types, and reference_list) can contain an arbitrary number of elements, depending on the number of prompts and their corresponding information.
Users building JSON files should follow the exact structure and field names as specified in the schema to ensure compatibility and proper data representation within the app.

CSV Schema for the Data:

Prompts,Types,Reference List
string,string,string
...


Explanation:

Prompts (string): This column should contain the prompts that represent different types of questions or scenarios for the model to answer or process.
Types (string): This column should contain the types corresponding to each prompt. These types categorize the prompts based on the nature of the question or scenario, such as "Knowledge Retrieval," "Logical Reasoning," etc.
Reference List (string): This column should contain the example reference answers for each prompt. These reference answers represent the expected model responses to the corresponding prompts.
Note:

The ... in the schema indicates that there can be an arbitrary number of rows in the CSV file, each corresponding to a different prompt, type, and reference answer.
Users building CSV files should strictly follow the order of columns as specified in the schema to ensure proper mapping and accuracy. Each row in the CSV should contain data for one prompt, its corresponding type, and the reference answer.

In [ ]:
# Load data from JSON file into Dictionary

import json

# Load data from the JSON file into a dictionary
with open("data.json", "r") as json_file:
    data = json.load(json_file)

# Print the dictionary to check the loaded data
print(data)


{'user_id': 'jbottum', 'project_id': 'project1', 'model_id': 'google/flan-t5-large', 'prompts': ['What is the capital of Germany?', 'What is the capital of Spain?', 'What is the capital of Canada?', 'What is the next number in the sequence: 2, 4, 6, 8, ...? If all cats have tails, and Fluffy is a cat, does Fluffy have a tail?', 'If you eat too much junk food, what will happen to your health? How does smoking affect the risk of lung cancer?', 'In the same way that pen is related to paper, what is fork related to? If tree is related to forest, what is brick related to?', 'Every time John eats peanuts, he gets a rash. Does John have a peanut allergy? Every time Sarah studies for a test, she gets an A. Will Sarah get an A on the next test if she studies?', 'All dogs have fur. Max is a dog. Does Max have fur? If it is raining outside, and Mary does not like to get wet, will Mary take an umbrella?', 'If I had studied harder, would I have passed the exam? What would have happened if Thomas Ed

In [ ]:
import time
import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline, HuggingFacePipeline

# Load tokenizer
tokenizer_start_time = time.time()
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer_end_time = time.time()

# Load model
model_start_time = time.time()
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
model_end_time = time.time()

# Load pipeline
pipe_start_time = time.time()
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
local_llm = HuggingFacePipeline(pipeline=pipe)
pipe_end_time = time.time()

# Store loading times
model_load_time = model_end_time - model_start_time
tokenizer_load_time = tokenizer_end_time - tokenizer_start_time
pipeline_load_time = pipe_end_time - pipe_start_time

In [13]:
import time
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, pipeline
import bert_score

# Create the pipeline for text generation
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=50)

# Initialize the dictionary to store results
epoch_record = {}

# Loop through the prompts and generate text
for i, prompt in enumerate(data['prompts']):
    start_time = time.time()
    generated = pipe(prompt, num_return_sequences=1)[0]['generated_text']
    end_time = time.time()
    generation_time = end_time - start_time

    # Store the results in a dictionary record
    record = {
        'timestamp': start_time,
        'project_name': data['project_id'],
        'user_id': data['user_id'],
        'model_id': data['model_id'],
        'prompt': prompt,
        'type': data['types'][i],
        'reference_answer': data['reference_list'][i],
        'generated_answer': generated,
        'generation_time': generation_time,
        'model_load_time': model_load_time,
        'tokenizer_load_time': tokenizer_load_time,
        'pipeline_load_time': pipeline_load_time,
    }

    # Calculate BERTScore
    P, R, F1 = bert_score.score([generated], [record['reference_answer']], lang="en", verbose=False)
    record['precision'] = P.numpy()[0].item()
    record['recall'] = R.numpy()[0].item()
    record['f1'] = F1.numpy()[0].item()

    # Store the record in epoch_record with a unique key
    epoch_record[f"Example {i + 1}"] = record



In [ ]:
# Print the results in the same format as the current script
import time
import datetime
from transformers import pipeline
import bert_score
print()
print(f"Model: {data['model_id']}")
print(f"Project: {data['project_id']}")
print(f"User: {data['user_id']}")
print("Run Date:", datetime.datetime.fromtimestamp(record['timestamp']).strftime('%Y-%m-%d %H:%M:%S'))
print("Model load time:", round(record['model_load_time'], 5))
print("Tokenizer load time:", round(record['tokenizer_load_time'], 5))
print("Pipeline load time:", round(record['pipeline_load_time'], 5))
print("=" * 30)

for example_number, record in epoch_record.items():
    print(f"{example_number}:")
    print("Prompt:", record['prompt'])
    print("Generated Text:", record['generated_answer'])
    print("Reference Answer:", record['reference_answer'])
    print("Generation Time:", round(record['generation_time'], 5))

    print("Type:", record['type'])
    print("Precision:", round(record['precision'], 5))
    print("Recall:", round(record['recall'], 5))
    print("F1 Score:", round(record['f1'], 5))
    print("=" * 10)


Model: google/flan-t5-large
Project: project1
User: jbottum
Run Date: 2023-07-21 16:26:06
Model load time: 68.148
Tokenizer load time: 1.9534
Pipeline load time: 0.18197
Example 1:
Prompt: What is the capital of Germany?
Generated Text: berlin
Reference Answer: The capital of Germany is Berlin
Generation Time: 1.45771
Type: Knowledge Retrieval
Precision: 0.80484
Recall: 0.82159
F1 Score: 0.81313
Example 2:
Prompt: What is the capital of Spain?
Generated Text: turin
Reference Answer: The capital of Spain is Madrid
Generation Time: 1.39184
Type: Knowledge Retrieval
Precision: 0.76089
Recall: 0.79643
F1 Score: 0.77825
Example 3:
Prompt: What is the capital of Canada?
Generated Text: toronto
Reference Answer: The capital of Canada is Ottawa
Generation Time: 1.64904
Type: Knowledge Retrieval
Precision: 0.77874
Recall: 0.78587
F1 Score: 0.78229
Example 4:
Prompt: What is the next number in the sequence: 2, 4, 6, 8, ...? If all cats have tails, and Fluffy is a cat, does Fluffy have a tail?
G

In [12]:
import csv
import json

# Save epoch_record as a CSV file
csv_file = "epoch_record.csv"
with open(csv_file, "w", newline="") as csvfile:
    fieldnames = [
        'timestamp', 'project_name', 'user_id', 'model_id', 'prompt', 'type',
        'reference_answer', 'generated_answer', 'generation_time',
        'model_load_time','tokenizer_load_time','pipeline_load_time',
        'precision', 'recall', 'f1'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for example_number, record in epoch_record.items():
        writer.writerow(record)

# Save epoch_record as a JSON file
json_file = "epoch_record.json"
with open(json_file, "w") as jsonfile:
    json.dump(epoch_record, jsonfile, indent=4)


NameError: ignored

In [11]:
import pandas as pd

# Read the CSV file
csv_file = "epoch_record.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame with headers
df


FileNotFoundError: ignored

In [ ]:
import json
from pprint import pprint

# Read the JSON file
json_file = "epoch_record.json"
with open(json_file, "r") as jsonfile:
    json_data = json.load(jsonfile)

# Display the JSON data with formatting
pprint(json_data)


{'Example 1': {'f1': 0.8131290674209595,
               'generated_answer': 'berlin',
               'generation_time': 1.4577090740203857,
               'model_id': 'google/flan-t5-large',
               'model_load_time': 68.14800214767456,
               'pipeline_load_time': 0.1819748878479004,
               'precision': 0.8048427104949951,
               'project_name': 'project1',
               'prompt': 'What is the capital of Germany?',
               'recall': 0.821587860584259,
               'reference_answer': 'The capital of Germany is Berlin',
               'timestamp': 1689956682.7405634,
               'tokenizer_load_time': 1.9533982276916504,
               'type': 'Knowledge Retrieval',
               'user_id': 'jbottum'},
 'Example 10': {'f1': 0.8048513531684875,
                'generated_answer': 'about 425 km/230 nm to the west of Fort '
                                    'Myers in Florida, and it is about 550 '
                                    'km/297 n

In [ ]:
# Code to import MMLU files and test and score those prompts and choices verses the refrence answers provided.

# Step 1 - import MMLU CSV files into a python dictionary

In [6]:
import pandas as pd

# Load MMLU dataset from CSV without headers
# mmlu_dataset_csv = "/content/formal_logic_dev.csv"
mmlu_dataset_csv = "/content/high_school_world_history_dev.csv"
# mmlu_dataset_csv = "/content/high_school_geography_dev.csv"
column_names = ['prompt', 'choiceA', 'choiceB', 'choiceC', 'choiceD', 'reference_answer']
df_mmlu = pd.read_csv(mmlu_dataset_csv, header=None, names=column_names)


df_mmlu


,prompt,choiceA,choiceB,choiceC,choiceD,reference_answer
0,This question refers to the following informat...,Hinduism,Buddhism,Shintoism,Zoroastrianism,A
1,This question refers to the following informat...,The Industrial Revolution,Decolonization,Regional Free Trade Associations,Autarky,B
2,This question refers to the following informat...,Social acceptance of child labor,Declining life expectancy in Germany,Criticisms of German trade tariffs,Negative effects attributed to industrial capi...,D
3,This question refers to the following informat...,The maintenance of military supremacy at all c...,Expanding tensions between religious sects,Factors that brought about the collapse of the...,Peacemaking efforts among the Islamic empires,B
4,This question refers to the following informat...,A breakdown in trade routes through the collap...,An increase in the population of the world thr...,The spread of Chinese and Indian belief system...,An increase in social unrest,B


In [14]:

# Create the modified data dictionary to store the MMLU data
data_mmlu = {}

# Extract prompts, choices, and reference answers from the dataset
prompts_mmlu = df_mmlu['prompt'].tolist()
choices_A_mmlu = df_mmlu['choiceA'].tolist()
choices_B_mmlu = df_mmlu['choiceB'].tolist()
choices_C_mmlu = df_mmlu['choiceC'].tolist()
choices_D_mmlu = df_mmlu['choiceD'].tolist()
reference_list_mmlu = df_mmlu[column_names[-1]].tolist()  # Assuming the last column contains reference answers
# reference_list_mmlu = df_mmlu['reference answer'].tolist()

# Populate the modified data dictionary with MMLU data
data_mmlu['user_id'] = 'mmlu_user'
data_mmlu['project_id'] = 'mmlu_project'
data_mmlu['model_id'] = 'mmlu_model'
data_mmlu['prompts'] = prompts_mmlu
data_mmlu['types'] = ['Unknown'] * len(prompts_mmlu)  # Replace 'Unknown' with the actual types for the MMLU dataset
data_mmlu['reference_list'] = [''] * len(prompts_mmlu)  # Placeholder for MMLU since class labels are in choices

# Map the class labels to the reference answers and store in the data dictionary
class_label_to_reference = {'A': 'choiceA', 'B': 'choiceB', 'C': 'choiceC', 'D': 'choiceD'}
for i, reference_label in enumerate(reference_list_mmlu):
    reference_choice = class_label_to_reference[reference_label]
    data_mmlu['reference_list'][i] = df_mmlu[reference_choice].iloc[i]

# Now use the MMLU prompts and reference answers in your script
# For example:
# prompts = data_mmlu['prompts']
# reference_list = data_mmlu['reference_list']


In [27]:
print (data_mmlu['prompts'])
print()
print (data_mmlu['reference_list'])

['This question refers to the following information.\nHe contains all works and desires and all perfumes and all tastes. He enfolds the whole universe and in silence is loving to all. This is the Spirit that is in my heart, this is Brahman. To him I shall come when I go beyond this life, and to him will come he who has faith and doubts not.\n—The Upanishads, India, c. 1000 BCE\nTo which religion does the speaker most likely belong? [MASK] Hinduism [SEP] Buddhism [SEP] Shintoism [SEP] Zoroastrianism [SEP]', 'This question refers to the following information.\n"The struggle against neo-colonialism is not aimed at excluding the capital of the developed world from operating in less developed countries. It is aimed at preventing the financial power of the developed countries being used in such a way as to impoverish the less developed.\nNon-alignment, as practiced by Ghana and many other countries, is based on co-operation with all States whether they be capitalist, socialist or have a mixe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
user_id = 'jbottum'
project_id = 'project1'
model_id = 'google/flan-t5-large'

In [18]:
prompts = data_mmlu['prompts']
reference_list = data_mmlu['reference_list']
# types = 'MMLU high school geography dev'
types = mmlu_dataset_csv

In [19]:
# load sample model data into a python dictionary
data = {
    "user_id": user_id,
    "project_id": project_id,
    "model_id": model_id,
    "prompts": prompts,
    "types": types,
    "reference_list": reference_list
}


In [24]:
import time
import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline, HuggingFacePipeline

# Load tokenizer
tokenizer_start_time = time.time()
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer_end_time = time.time()

# Load model
model_start_time = time.time()
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
model_end_time = time.time()

# Load pipeline
pipe_start_time = time.time()
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
local_llm = HuggingFacePipeline(pipeline=pipe)
pipe_end_time = time.time()

# Store loading times
model_load_time = model_end_time - model_start_time
tokenizer_load_time = tokenizer_end_time - tokenizer_start_time
pipeline_load_time = pipe_end_time - pipe_start_time

In [25]:
import time
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, pipeline
import bert_score

# Create the pipeline for text generation
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=50)

# Initialize the dictionary to store results
epoch_record = {}

# Loop through the prompts and generate text
for i, prompt in enumerate(data['prompts']):
    start_time = time.time()
    generated = pipe(prompt, num_return_sequences=1)[0]['generated_text']
    end_time = time.time()
    generation_time = end_time - start_time

    # Store the results in a dictionary record
    record = {
        'timestamp': start_time,
        'project_name': data['project_id'],
        'user_id': data['user_id'],
        'model_id': data['model_id'],
        'prompt': prompt,
        'type': mmlu_dataset_csv,
        'reference_answer': data['reference_list'][i],
        'generated_answer': generated,
        'generation_time': generation_time,
        'model_load_time': model_load_time,
        'tokenizer_load_time': tokenizer_load_time,
        'pipeline_load_time': pipeline_load_time,
    }

    # Calculate BERTScore
    P, R, F1 = bert_score.score([generated], [record['reference_answer']], lang="en", verbose=False)
    record['precision'] = P.numpy()[0].item()
    record['recall'] = R.numpy()[0].item()
    record['f1'] = F1.numpy()[0].item()

    # Store the record in epoch_record with a unique key
    epoch_record[f"Example {i + 1}"] = record

In [26]:
# Print the results in the same format as the current script
import time
import datetime
from transformers import pipeline
import bert_score
print()
print(f"Model: {data['model_id']}")
print(f"Project: {data['project_id']}")
print(f"User: {data['user_id']}")
print("Run Date:", datetime.datetime.fromtimestamp(record['timestamp']).strftime('%Y-%m-%d %H:%M:%S'))
print("Model load time:", round(record['model_load_time'], 5))
print("Tokenizer load time:", round(record['tokenizer_load_time'], 5))
print("Pipeline load time:", round(record['pipeline_load_time'], 5))
print("=" * 30)

for example_number, record in epoch_record.items():
    print(f"{example_number}:")
    print("Prompt:", record['prompt'])
    print("Generated Text:", record['generated_answer'])
    print("Reference Answer:", record['reference_answer'])
    print("Generation Time:", round(record['generation_time'], 5))

    print("Type:", record['type'])
    print("Precision:", round(record['precision'], 5))
    print("Recall:", round(record['recall'], 5))
    print("F1 Score:", round(record['f1'], 5))
    print("=" * 10)


Model: google/flan-t5-large
Project: project1
User: jbottum
Run Date: 2023-07-23 15:00:50
Model load time: 30.59886
Tokenizer load time: 0.18311
Pipeline load time: 0.00608
Example 1:
Prompt: This question refers to the following information.
He contains all works and desires and all perfumes and all tastes. He enfolds the whole universe and in silence is loving to all. This is the Spirit that is in my heart, this is Brahman. To him I shall come when I go beyond this life, and to him will come he who has faith and doubts not.
—The Upanishads, India, c. 1000 BCE
To which religion does the speaker most likely belong?
Generated Text: Hinduism
Reference Answer: Hinduism
Generation Time: 1.96042
Type: /content/high_school_world_history_dev.csv
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Example 2:
Prompt: This question refers to the following information.
"The struggle against neo-colonialism is not aimed at excluding the capital of the developed world from operating in less developed countri

In [23]:
import pandas as pd

# Function to build the multiple-choice prompt
def build_multiple_choice_prompt(question, choices):
    prompt = f"{question} [MASK] "
    for choice in choices:
        prompt += f"{choice} [SEP] "
    return prompt.strip()

# Load MMLU dataset from CSV without headers
mmlu_dataset_csv = "/content/high_school_world_history_dev.csv"
column_names = ['prompt', 'choiceA', 'choiceB', 'choiceC', 'choiceD', 'reference_answer']
df_mmlu = pd.read_csv(mmlu_dataset_csv, header=None, names=column_names)

# Create the modified data dictionary to store the MMLU data
data_mmlu = {}

# Extract prompts, choices, and reference answers from the dataset
prompts_mmlu = df_mmlu['prompt'].tolist()
choices_A_mmlu = df_mmlu['choiceA'].tolist()
choices_B_mmlu = df_mmlu['choiceB'].tolist()
choices_C_mmlu = df_mmlu['choiceC'].tolist()
choices_D_mmlu = df_mmlu['choiceD'].tolist()
reference_list_mmlu = df_mmlu[column_names[-1]].tolist()  # Assuming the last column contains reference answers

# Populate the modified data dictionary with MMLU data
data_mmlu['user_id'] = 'mmlu_user'
data_mmlu['project_id'] = 'mmlu_project'
data_mmlu['model_id'] = 'mmlu_model'
data_mmlu['prompts'] = prompts_mmlu
data_mmlu['types'] = ['Unknown'] * len(prompts_mmlu)  # Replace 'Unknown' with the actual types for the MMLU dataset
data_mmlu['reference_list'] = [''] * len(prompts_mmlu)  # Placeholder for MMLU since class labels are in choices

# Map the class labels to the reference answers and store in the data dictionary
class_label_to_reference = {'A': 'choiceA', 'B': 'choiceB', 'C': 'choiceC', 'D': 'choiceD'}
for i, reference_label in enumerate(reference_list_mmlu):
    reference_choice = class_label_to_reference[reference_label]
    data_mmlu['reference_list'][i] = df_mmlu[reference_choice].iloc[i]

# Now, for each prompt, create the multiple-choice prompt
multiple_choice_prompts = []
for i, prompt in enumerate(data_mmlu['prompts']):
    choices = [choices_A_mmlu[i], choices_B_mmlu[i], choices_C_mmlu[i], choices_D_mmlu[i]]
    multiple_choice_prompt = build_multiple_choice_prompt(prompt, choices)
    multiple_choice_prompts.append(multiple_choice_prompt)

# Replace the original prompts with the multiple-choice prompts
data_mmlu['prompts'] = multiple_choice_prompts

# Now you can use the modified data_mmlu dictionary in your script with the multiple-choice prompts and reference answers.


In [28]:
print (data_mmlu['prompts'])

['This question refers to the following information.\nHe contains all works and desires and all perfumes and all tastes. He enfolds the whole universe and in silence is loving to all. This is the Spirit that is in my heart, this is Brahman. To him I shall come when I go beyond this life, and to him will come he who has faith and doubts not.\n—The Upanishads, India, c. 1000 BCE\nTo which religion does the speaker most likely belong? [MASK] Hinduism [SEP] Buddhism [SEP] Shintoism [SEP] Zoroastrianism [SEP]', 'This question refers to the following information.\n"The struggle against neo-colonialism is not aimed at excluding the capital of the developed world from operating in less developed countries. It is aimed at preventing the financial power of the developed countries being used in such a way as to impoverish the less developed.\nNon-alignment, as practiced by Ghana and many other countries, is based on co-operation with all States whether they be capitalist, socialist or have a mixe